In [1]:
# data processing packages
import numpy as np   
import pandas as pd 
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *
 
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import IPython
from IPython.display import display

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')
import itertools

import random

# machine leanring packages

from keras.models import Sequential 
from keras.layers.core import Dense, Activation, TimeDistributedDense, Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import *
from keras.callbacks import *
# RMSprop, Adadelta
from keras.regularizers import l2, activity_l2
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score



Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# ------------data prepro-----------------------

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

print xtrain_df.shape, xtest_df.shape, ytrain_df.shape, ytest_df.shape


# get training and testing data prepared

# cxtrain, dxtrain = conti_cate_split(xtrain_df, 106)
# cxtest, dxtest = conti_cate_split(xtest_df, 106)

# cxtest = conti_normalization_test_dta(cxtest, cxtrain)
# cxtrain = conti_normalization_train_dta(cxtrain)

print xtrain_df.shape, ytrain_df.shape
print xtest_df.shape,  ytest_df.shape

# cxtrain = cxtrain.as_matrix() 
xtrain = xtrain_df.as_matrix()
xtest  = xtest_df.as_matrix()

ytrain = ytrain_df.as_matrix()
ytest  = ytest_df.as_matrix()

# ytest = np_utils.to_categorical( ytest_df.as_matrix() )
# ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

(6938, 186) (771, 186) (6938, 1) (771, 1)
(6938, 186) (6938, 1)
(771, 186) (771, 1)


In [18]:
def crossv_rf_num_estimatior(maxnum, X,Y):
    score_list=[]
    tmpscore=0
    tmpnum=0
    
    tmpy = Y.reshape( (len(Y),) )
    
    for i in range(1,maxnum+1,10):
        clf = RandomForestClassifier(n_estimators=i,max_depth=3)
        scores = cross_val_score(clf, X, tmpy)
        score_list.append(scores)
    
        if scores.mean()>tmpscore:
            tmpscore=scores.mean()
            tmpnum=i
    return tmpnum, score_list

def crossv_gbt_num_estimatior(maxnum,X,Y):
    score_list=[]
    tmpscore=0
    tmpnum=0
    
    tmpy = Y.reshape( (len(Y),) )
    
    for i in range(1,maxnum+1,10):
        clf = GradientBoostingClassifier(n_estimators=i,learning_rate=0.1)
        scores = cross_val_score(clf, X, tmpy)
        score_list.append(scores)
    
        if scores.mean()>tmpscore:
            tmpscore=scores.mean()
            tmpnum=i
    return tmpnum, score_list

In [16]:
rf_num, rf_slist= crossv_rf_num_estimatior(100,xtrain,ytrain)
rf=RandomForestClassifier(n_estimators=rf_num,max_depth=3).fit(xtrain,ytrain)
print "# trees:",rf_num
print "Random forest test accuracy:",rf.score(xtest,ytest)
print "Random forest training accuracy:",rf.score(xtrain,ytrain)


# trees: 11
Random forest test accuracy: 0.294422827497
Random forest training accuracy: 0.30729316806


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


In [19]:
gbt_num, gbt_slist=crossv_gbt_num_estimatior(100,xtrain,ytrain)
gbt = GradientBoostingClassifier(n_estimators=gbt_num,learning_rate=0.1).fit(xtrain,ytrain)
print "# iterations:",gbt_num
print "Gradient boosting tree test accuracy:", gbt.score(xtest,ytest)
print "Gradient boosting tree training accuracy:", gbt.score(xtrain,ytrain)


# iterations: 21
Gradient boosting tree test accuracy: 0.306095979248
Gradient boosting tree training accuracy: 0.429374459498


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
def result_plot( accu_list, feature_name, maxnum, fig_title, filename ):
    fig = plt.figure()
    
    figsize=( 15.4,7)
    font_size=15
    fig.set_size_inches( figsize )
    matplotlib.rcParams.update({'font.size': font_size})

    axes = plt.gca()
    axes.set_ylim([0.2,0.35])
    
    x= range(1,maxnum+1,10)
    
    for i in range(len(accu_list)):
        
        tmp=[]
        for j in range(len(accu_list[i])):
            tmp.append( accu_list[i][j].mean() )
        
        plt.plot( x, tmp ,label=feature_name[i] )

    plt.title(fig_title)
    plt.ylabel('Accuray on validation datasets')
    plt.xlabel('# of tress or iterations')
    # plt.legend( loc='upper left',fontsize=12 )
    plt.legend(loc='upper right')

    fig.savefig('./results/'+filename+'.jpg', format='jpg', bbox_inches='tight')

def feature_importance_plot(feature_import, title, filename, yrange):
    
    fig=plt.figure() 
    y_pos = np.arange(len(feature_import))
    
    axes = plt.gca()
    axes.set_xlim([0,len(feature_import)+1])
    axes.set_ylim(yrange)

    
    plt.bar(y_pos, feature_import, align='center', alpha=0.5)
# plt.xticks(y_pos, label_set)
    plt.ylabel('Feature importance weights \n (sum of weights =1)')
    plt.xlabel('Features (the same order as in the data file)')

    plt.title(title)
    
    fig.savefig('./results/'+filename+'.jpg', format='jpg', bbox_inches='tight')
    
def total_feature_importance_plot(feature_import, title, filename):
    
    fig=plt.figure() 
    y_pos = np.arange(len(feature_import))
    
    axes = plt.gca()
    axes.set_xlim([-1,len(feature_import)+1])
    axes.set_ylim([0,0.5])

    
    plt.bar(y_pos, feature_import, align='center', alpha=0.5)
    plt.xticks(y_pos, ['Mobility','Frequency','Recency'])
    plt.ylabel('Feature importance weights \n (sum of weights =1)')
    plt.xlabel('Features groups')

    plt.title(title)
    
    fig.savefig('./results/'+filename+'.jpg', format='jpg', bbox_inches='tight')

